In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [2]:
# %%capture
# !pip install pandas tiktoken graphrag

In [3]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

## Global Search example

Global search method generates answers by searching over all AI-generated community reports in a map-reduce fashion. This is a resource-intensive method, but often gives good responses for questions that require an understanding of the dataset as a whole (e.g. What are the most significant values of the herbs mentioned in this notebook?).

### LLM setup

In [3]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

### Load community reports as context for global search

- Load all community reports in the `create_final_community_reports` table from the ire-indexing engine, to be used as context data for global search.
- Load entities from the `create_final_nodes` and `create_final_entities` tables from the ire-indexing engine, to be used for calculating community weights for context ranking. Note that this is optional (if no entities are provided, we will not calculate community weights and only use the `rank` attribute in the community reports table for context ranking)

In [4]:
# parquet files generated from indexing pipeline
INPUT_DIR = "output/20240716-183418/artifacts/"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [5]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 46


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,44,# Devi and her Relationships\n\nThe community ...,2,8.5,Devi and her Relationships,The impact severity rating is high due to the ...,"The community revolves around Devi, a central ...",[{'explanation': 'Devi shares a close mother-d...,"{\n ""title"": ""Devi and her Relationships"",\...",44715762-c11e-43c9-8c26-3b5f969214b3
1,45,# Vaishali and Devi\n\nThe community revolves ...,2,7.5,Vaishali and Devi,The impact severity rating is high due to the ...,The community revolves around Vaishali and Dev...,[{'explanation': 'Vaishali believes Devi to be...,"{\n ""title"": ""Vaishali and Devi"",\n ""sum...",d66f8638-d285-4673-b0c2-86b408941209
2,10,# Devi and her Key Relationships\n\nThe commun...,1,8.5,Devi and her Key Relationships,The impact severity rating is high due to the ...,"The community revolves around Devi, a central ...",[{'explanation': 'Devi shares a close mother-d...,"{\n ""title"": ""Devi and her Key Relationship...",f464ff53-70eb-42b3-96f9-9718aaa591be
3,11,# RAM and Devi's Relationships\n\nThe communit...,1,8.5,RAM and Devi's Relationships,The impact severity rating is high due to the ...,The community revolves around the relationship...,[{'explanation': 'RAM and Devi share a close a...,"{\n ""title"": ""RAM and Devi's Relationships""...",6ad5bc84-4702-46ec-9f3c-3bee015bc0ec
4,12,# Characters and Locations in the Story\n\nThe...,1,7.5,Characters and Locations in the Story,The impact severity rating is high due to the ...,The community revolves around key characters s...,[{'explanation': 'Rackham plays a supportive r...,"{\n ""title"": ""Characters and Locations in t...",28912ca0-a989-4950-bf36-b29ad519436f


#### Build global context based on community reports

In [6]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [7]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [8]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [17]:
result = await search_engine.asearch(
    """
    List the key characters in this story together with their individual journeys. For each character, list the following: 
    (1) Character's background and context 
    (2) 3 - 5 key events that shape the character with detailed descriptsion of each event
    """
)

response = result.response

from IPython.display import display, HTML
display(HTML(f"<p style='width: 70ch;'>{response}</p>"))

In [12]:
print(response)

### Story Outline:

In this narrative, several key characters play pivotal roles, each contributing uniquely to the intricate tapestry of events and relationships within the community. These characters are central to the story's development and are essential in shaping the dynamics and outcomes of significant events.

#### Devi:
Devi stands as a central character with intricate relationships with entities like Sunaina, Sita, Rama, and others. Her emotional context involves love, conflict, trust, and vulnerability. Devi's beliefs revolve around loyalty, destiny, and the power of relationships. She seeks to fulfill her destiny and protect her loved ones. Devi's needs include guidance, acceptance, and understanding. Key events shaping Devi include her interactions with various characters like Sunaina, Sita, Rama, Hanuman, Meena, Urmila, Ram, and Agni Devata, culminating in her transformation into the fire lord with the assistance of Agni Devata.

#### Sita:
Sita, another central character

In [19]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,118,AI and Technology Ecosystem,0.154294,# AI and Technology Ecosystem\n\nThe community...,8.5
1,99,Apple and AI Ecosystem,0.144105,# Apple and AI Ecosystem\n\nThe community revo...,8.5
2,115,Nikhil and AI Ecosystem,0.101892,# Nikhil and AI Ecosystem\n\nThe community rev...,8.5
3,39,"Adobe, Microsoft Azure, and AWS Community",0.040757,"# Adobe, Microsoft Azure, and AWS Community\n\...",8.5
4,52,META and Mark Zuckerberg,0.034934,# META and Mark Zuckerberg\n\nThe community re...,8.5
...,...,...,...,...,...
101,10,Human Intelligence and New Situations,0.001456,# Human Intelligence and New Situations\n\nThe...,7.5
102,103,Sam Altman and John Ivy in the AI Industry,0.005822,# Sam Altman and John Ivy in the AI Industry\n...,7.5
103,17,AI Task and Email Classification,0.004367,# AI Task and Email Classification\n\nThe comm...,7.5
104,64,Google Colab and Cloud Platforms,0.002911,# Google Colab and Cloud Platforms\n\nThe comm...,7.5


In [20]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 6. LLM tokens: 56271


# Graph viz


## yfiles_jupyter_graphs

Source: https://github.com/microsoft/graphrag/issues/418

In [1]:
%%capture
!pip install yfiles_jupyter_graphs pandas notebook ipywidgets

In [1]:
"""
Uses yfiles-jupyter-graphs to visualize the dataframes.

The dataframes are converted into supported nodes and relationships lists and then passed to yfiles-jupyter-graphs.
Additionally, some values are mapped to visualization properties.
"""

import pandas as pd

def show_graph(entity_df, relationship_df):
    from yfiles_jupyter_graphs import GraphWidget

    # converts the entities dataframe to a list of dicts for yfiles-jupyter-graphs
    def convert_entities_to_dicts(df):
        nodes_dict = {}
        for index, row in df.iterrows():
            # Create a dictionary for each row and collect unique nodes
            node_id = row["title"]
            if not node_id in nodes_dict:
                nodes_dict[node_id] = {"id": node_id, "properties": {col: value for col, value in zip(row.index, row.values)}}
        return list(nodes_dict.values())

    # converts the relationships dataframe to a list of dicts for yfiles-jupyter-graphs
    def convert_relationships_to_dicts(df):
        relationships = []
        for index, row in df.iterrows():
            # Create a dictionary for each row
            relationships.append({"start": row['source'], "end": row['target'], "properties": {col: value for col, value in zip(row.index, row.values)}})
        return relationships


    w = GraphWidget()
    # use the converted data to visualize the graph
    w.nodes = convert_entities_to_dicts(entity_df)
    w.edges = convert_relationships_to_dicts(relationship_df)
    w.directed = True
    # show title on the node
    w.node_label_mapping = "title"
    # map community to a color
    def community_to_color(community):
        colors = ["crimson", "darkorange", "indigo", "cornflowerblue", "cyan", "teal", "green"]
        return colors[int(community) % len(colors)] if community is not None else "lightgray"
    def edge_to_source_community(edge):
        source_node = next((entry for entry in w.nodes if entry["properties"]["title"] == edge["start"]), None)
        source_node_community = source_node["properties"]["community"]
        return source_node_community if source_node_community is not None else None
    w.node_color_mapping = lambda node : community_to_color(node["properties"]["community"])
    w.edge_color_mapping = lambda edge : community_to_color(edge_to_source_community(edge))
    # map size data to a reasonable factor
    w.node_scale_factor_mapping = lambda node : 0.5 + node["properties"]["size"] * 1.5 / 20
    # use weight for edge thickness
    w.edge_thickness_factor_mapping = "weight"
    # Use the circular layout for this visualization. For larger graphs, the default organic layout is often preferable.
    w.circular_layout()
    display(w)

# pass the dataframes from the parquet files
entity_df = pd.read_parquet("/workspaces/graphrag/nm-projects/asai/output/20240716-171659/artifacts/create_final_nodes.parquet")
relationship_df = pd.read_parquet("/workspaces/graphrag/nm-projects/asai/output/20240716-171659/artifacts/create_final_relationships.parquet")

show_graph(entity_df, relationship_df)

GraphWidget(layout=Layout(height='800px', width='100%'))

## pyvis (not as good as yfiles_jupyter_graphs)

In [ ]:
!pip install pandas networkx pyvis

In [ ]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

df = pd.read_parquet('/workspaces/graphrag/nm-projects/asai/output/20240716-171659/artifacts/create_base_entity_graph.parquet')
graphml_data = df.iloc[0]['clustered_graph']
graph = nx.parse_graphml(graphml_data.encode('utf-8'))

net = Network(notebook=True)
net.from_nx(graph)
net.show('graph.html')